<a href="https://colab.research.google.com/github/bathlarajat/chartjsExamples/blob/master/Copy_of_Stock_Goli_Indicator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=5bb2950e99ef3aa4fa99cfc13c166c80cdc47af74b70e1415bb59993bd1f6dec
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [ ]:
# Import libraries and modules
import pandas as pd
import numpy as np
#import talib as ta
import pandas_ta as ta
import matplotlib.pyplot as plt
import yfinance as yf
from pandas_datareader.data import DataReader
from pandas_datareader import data as pdr
from datetime import datetime

import plotly.graph_objs as go
import plotly
import numpy as np
from plotly.subplots import make_subplots

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [ ]:
# Calculate the donchian channel function
def donchian(leng):
    return (df['Low'].rolling(leng).min() + df['High'].rolling(leng).max()) / 2
# Define the FRAMA function
def frama(price, leng, FC, SC):
    len1 = leng // 2
    w = np.log(2 / (SC + 1))
    H1 = df['High'].rolling(len1).max()
    L1 = df['Low'].rolling(len1).min()
    N1 = (H1 - L1) / len1
    H2 = df['High'].rolling(leng).max().shift(len1)
    L2 = df['Low'].rolling(leng).min().shift(len1)
    N2 = (H2 - L2) / len1
    H3 = df['High'].rolling(leng).max()
    L3 = df['Low'].rolling(leng).min()
    N3 = (H3 - L3) / leng
    dimen1 = (np.log(N1 + N2) - np.log(N3)) / np.log(2)
    dimen = np.where((N1 > 0) & (N2 > 0) & (N3 > 0), dimen1, dimen1.shift(1))
    alpha1 = np.exp(w * (dimen - 1))
    oldalpha = np.where(alpha1 > 1, 1, np.where(alpha1 < 0.01, 0.01, alpha1))
    oldN = (2 - oldalpha) / oldalpha
    N = (((SC - FC) * (oldN - 1)) / (SC - 1)) + FC
    alpha_ = 2 / (N + 1)
    alpha = np.where(alpha_ < 2 / (SC + 1), 2 / (SC + 1), np.where(alpha_ > 1, 1, alpha_))
    out = (1 - alpha) * L1 + alpha * price # Use the previous value of out as the initial value
    return out

In [ ]:

def calc_function(df):
    # Define parameters and variables
    conversionPeriods = 9
    basePeriods = 26
    laggingSpan2Periods = 52
    displacement = 26

    slen = 60
    llen = 375
    sper = "D"
    lper = "12M"
    pper = "W"

    periods = 51 # Moving average period

    FC = 11
    SC = 31 # Input value

    leng=17
    len5 = 17 # Input value
    FC5 = 21
    SC5 = 17 # Input value

    vwaplength = 1
    emaLength1 = 3
    avlength = 26

    # Get the source data from TradingView
    # You can use the ccxt library to connect to an exchange and get the data
    # For simplicity, we assume the data is already in a pandas dataframe called df
    # The dataframe has columns: open, high, low, close, volume


    # Calculate the conversion line, base line, lead line 1 and lead line 2
    conversionLine = donchian(conversionPeriods)
    baseLine = donchian(basePeriods)
    leadLine1 = (conversionLine + baseLine) / 2
    leadLine2 = donchian(laggingSpan2Periods)

    # Plot the lead line 1 with displacement
    #plt.plot(leadLine1.shift(displacement - 1), color='#A5D6A7', linewidth=5, label='Lead 1')

    # Check if the time frame is intraday or not
    isintraday = df.index.freq != 'D'

    # Get the start time of the current period
    per = sper if isintraday else lper
    start = df.index.to_period(per).to_timestamp()

    # Calculate the VWAP and the EMA of VWAP
    cvwap = ta.ema(df['Volume'] * (df['High'] + df['Low'] + df['Close']) / 3, vwaplength)
    plotvwap1 = cvwap # This is the VWAP to plot

    # Calculate the EMAs of close
    av11 = ta.ema(df['Close'], 8)
    av21 = ta.ema(df['Close'], 21)
    av31 = ta.ema(df['Close'], 170)

    # Plot the EMA of close with 170 periods
    #plt.plot(av31, color='fuchsia', linewidth=10, label='EMA 170')

    # Calculate the price as the average of high and low
    price = (df['High'] + df['Low']) / 2

    # Calculate the FRAMA values with different parameters
    out = frama(price, leng, FC, SC)
    out5 = frama(price, len5, FC5, SC5)

    # Plot the FRAMA values with different offsets
    #plt.plot(out.shift(17), color='blue', linewidth=3, label='FRAMA')
    #plt.plot(out5.shift(21), color='red', linewidth=3, label='FRAMA')
    return av31,out5

In [ ]:
def make_plot(df,intval):
    fig = make_subplots(rows=2, cols=1,shared_xaxes=True)

    fig.update_layout(
        width=1200,
        height=1200
    )

    fig.update_layout(
        title=dict(text="Stock symbol = " + symbol + '; Interval =' + intval, font=dict(size=25), automargin=True, yref='paper')
    )

    # price Line
    fig.append_trace(
        go.Scatter(
            x=df.index,
            y=df['Open'],
            line=dict(color='#ff9900', width=1),
            name='Open',
            # showlegend=False,
            legendgroup='1',
        ), row=1, col=1
    )
    # Candlestick chart for pricing
    fig.append_trace(
        go.Candlestick(
            x=df.index,
            open=df['Open'],
            high=df['High'],
            low=df['Low'],
            close=df['Close'],
            increasing_line_color='#ff9900',
            decreasing_line_color='black',
            showlegend=False
        ), row=1, col=1
    )

    fig.append_trace(
        go.Scatter(
            x=df.index,
            y=av31,
            line=dict(color='teal',width=1),
            name='EMA_170_av31',
            # showlegend=False,
            legendgroup='1',
        ), row=1, col=1
    )
    #plt.plot(av31, color='teal', linewidth=3, label='FRAMA')

    fig.append_trace(
        go.Scatter(
            x=df.index,
            y=out5.shift(21),
            line=dict(color='red', width=1),
            name='FramaFucntion_windowSize21',
            # showlegend=False,
            legendgroup='1',
        ), row=1, col=1
    )
    #plt.plot(out5.shift(21), color='red', linewidth=3, label='FRAMA')


    # Fast Signal (%k)
    fig.append_trace(
        go.Scatter(
            x=df.index,
            y=df['MACD_15_86_22'],
            line=dict(color='#ff9900', width=2),
            name='macd',
            # showlegend=False,
            legendgroup='2',
        ), row=2, col=1
    )
    # Slow signal (%d)
    fig.append_trace(
        go.Scatter(
            x=df.index,
            y=df['MACDs_15_86_22'],
            line=dict(color='#000000', width=2),
            # showlegend=False,
            legendgroup='2',
            name='signal'
        ), row=2, col=1
    )
    # Colorize the histogram values
    colors = np.where(df['MACDh_15_86_22'] < 0, '#000', '#ff9900')
    # Plot the histogram
    fig.append_trace(
        go.Bar(
            x=df.index,
            y=df['MACD_15_86_22'],
            name='histogram',
            marker_color=colors,
        ), row=2, col=1
    )

    # Make it pretty
    layout = go.Layout(
        plot_bgcolor='#efefef',
        # Font Families
        font_family='Monospace',
        font_color='#000000',
        font_size=20,
        xaxis=dict(
            rangeslider=dict(
                visible=False
            )
        )
    )
    # Update options and show plot
    fig.update_layout(layout)
    fig.show()
  #  fig.write_image(symbol+"_"+intval+".png")
    return 0

In [ ]:
# Request historic pricing data via finance.yahoo.com API
symbols = ['BAC','V','NCLH','CPOAX']
intvals = ['1h','1d']

In [ ]:
for symbol in symbols:
    for intval in intvals:
        df = yf.Ticker(symbol).history(period='1y',interval=intval)[['Close', 'Open', 'High', 'Volume', 'Low']]
        # Calculate MACD values using the pandas_ta library
        df.ta.macd(close='Close', fast=15, slow=86, signal=22, append=True)
        av31,out5 = calc_function(df)
        make_plot(df,intval)
# View result#
#pd.set_option("display.max_columns", None)  # show all columns
#print(df)



<ipython-input-4-663d0b194be0>:48: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



<ipython-input-4-663d0b194be0>:48: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



<ipython-input-10-3e4b90a1fe73>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-4-663d0b194be0>:48: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



<ipython-input-4-663d0b194be0>:48: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



<ipython-input-4-663d0b194be0>:48: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



<ipython-input-4-663d0b194be0>:48: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



<ipython-input-4-663d0b194be0>:48: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



<ipython-input-4-663d0b194be0>:48: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



In [ ]:
df

,Close,Open,High,Volume,Low,MACD_15_86_22,MACDh_15_86_22,MACDs_15_86_22
Date,,,,,,,,
2023-02-13 00:00:00-05:00,20.290001,20.290001,20.290001,0,20.290001,NaN,NaN,NaN
2023-02-14 00:00:00-05:00,20.830000,20.830000,20.830000,0,20.830000,NaN,NaN,NaN
2023-02-15 00:00:00-05:00,22.280001,22.280001,22.280001,0,22.280001,NaN,NaN,NaN
2023-02-16 00:00:00-05:00,21.379999,21.379999,21.379999,0,21.379999,NaN,NaN,NaN
2023-02-17 00:00:00-05:00,20.750000,20.750000,20.750000,0,20.750000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2024-02-06 00:00:00-05:00,25.520000,25.520000,25.520000,0,25.520000,1.061186,-0.326302,1.387488
2024-02-07 00:00:00-05:00,25.950001,25.950001,25.950001,0,25.950001,1.116630,-0.247305,1.363935
2024-02-08 00:00:00-05:00,26.980000,26.980000,26.980000,0,26.980000,1.265892,-0.089518,1.355410


In [ ]:
import os

os.system('jupyter nbconvert --to html Goli_Indicator.ipynb')

256

In [ ]:
!jupyter nbconvert --to html Goli_Indicator.ipynb

Traceback (most recent call last):
  File "/opt/conda/bin/jupyter-nbconvert", line 7, in <module>
    from nbconvert.nbconvertapp import main
  File "/opt/conda/lib/python3.9/site-packages/nbconvert/nbconvertapp.py", line 140, in <module>
    class NbConvertApp(JupyterApp):
  File "/opt/conda/lib/python3.9/site-packages/nbconvert/nbconvertapp.py", line 225, in NbConvertApp
    """.format(formats=get_export_names()))
  File "/opt/conda/lib/python3.9/site-packages/nbconvert/exporters/base.py", line 141, in get_export_names
    e = get_exporter(exporter_name)(config=config)
  File "/opt/conda/lib/python3.9/site-packages/nbconvert/exporters/base.py", line 102, in get_exporter
    if getattr(exporter(config=config), 'enabled', True):
  File "/opt/conda/lib/python3.9/site-packages/nbconvert/exporters/templateexporter.py", line 325, in __init__
    super().__init__(config=config, **kw)
  File "/opt/conda/lib/python3.9/site-packages/nbconvert/exporters/exporter.py", line 114, in __init__
    s

import plotly.graph_objs as go
import plotly
import numpy as np
from plotly.subplots import make_subplots

#import plotly.express as plt
# Construct a 2 x 1 Plotly figure
fig = make_subplots(rows=2, cols=1,shared_xaxes=True)

fig.update_layout(
    width=1200,
    height=1200
)

fig.update_layout(
    title=dict(text="Stock symbol = " + symbol, font=dict(size=50), automargin=True, yref='paper')
)

# price Line
fig.append_trace(
    go.Scatter(
        x=df.index,
        y=df['Open'],
        line=dict(color='#ff9900', width=1),
        name='Open',
        # showlegend=False,
        legendgroup='1',
    ), row=1, col=1
)
# Candlestick chart for pricing
fig.append_trace(
    go.Candlestick(
        x=df.index,
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        increasing_line_color='#ff9900',
        decreasing_line_color='black',
        showlegend=False
    ), row=1, col=1
)

fig.append_trace(
    go.Scatter(
        x=df.index,
        y=av31,
        line=dict(color='teal',width=1),
        name='ShiftedPrice_av31',
        # showlegend=False,
        legendgroup='1',
    ), row=1, col=1
)
#plt.plot(av31, color='teal', linewidth=3, label='FRAMA')

fig.append_trace(
    go.Scatter(
        x=df.index,
        y=out5.shift(21),
        line=dict(color='red', width=1),
        name='ShiftedPrice_21',
        # showlegend=False,
        legendgroup='1',
    ), row=1, col=1
)
#plt.plot(out5.shift(21), color='red', linewidth=3, label='FRAMA')


# Fast Signal (%k)
fig.append_trace(
    go.Scatter(
        x=df.index,
        y=df['MACD_15_86_22'],
        line=dict(color='#ff9900', width=2),
        name='macd',
        # showlegend=False,
        legendgroup='2',
    ), row=2, col=1
)
# Slow signal (%d)
fig.append_trace(
    go.Scatter(
        x=df.index,
        y=df['MACD_15_86_22'],
        line=dict(color='#000000', width=2),
        # showlegend=False,
        legendgroup='2',
        name='signal'
    ), row=2, col=1
)
# Colorize the histogram values
colors = np.where(df['MACD_15_86_22'] < 0, '#000', '#ff9900')
# Plot the histogram
fig.append_trace(
    go.Bar(
        x=df.index,
        y=df['MACD_15_86_22'],
        name='histogram',
        marker_color=colors,
    ), row=2, col=1
)
# Make it pretty
layout = go.Layout(
    plot_bgcolor='#efefef',
    # Font Families
    font_family='Monospace',
    font_color='#000000',
    font_size=20,
    xaxis=dict(
        rangeslider=dict(
            visible=False
        )
    )
)
# Update options and show plot
fig.update_layout(layout)
fig.show()